In [10]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import sys
import numpy as np

# 建立鉴别器

In [11]:
def buile_discriminator():
    model =  Sequential()
    
    img_shape = (28, 28, 1)
    model.add(Flatten(input_shape = img_shape))
    
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    img = Input(shape=img_shape)
    validity = model(img)
    
    return Model(img, validity)

# 建立并编译鉴别器

In [12]:
optimizer = Adam(0.0002, 0.5)

discriminator = buile_discriminator()
discriminator.compile(loss='binary_crossentropy',
                     optimizer=optimizer,
                     metrics=['accuracy'])

# 该模型只有一个输入，用于输入判断该图片是真的还是假的

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params:

# 建立生成器

In [13]:
latent_dim = 100

def build_generator():
    
    model = Sequential()
    
    img_shape = (28, 28, 1)
    model.add(Dense(256, input_dim = latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    noise = Input(shape=(latent_dim, ))
    img = model(noise)
    
    return Model(noise, img)

# 堆叠 Generator 和 Discriminator

In [14]:
# 建立生成器
generator = build_generator()

# 使用噪音生成图片
z = Input(shape = (latent_dim, ))
img = generator(z)

# 只训练生成器
discriminator.trainable = False

# 将图片输入给辨别器辨别图片
validity = discriminator(img)

# 堆叠 generator 和 discriminator
# 建立生成器欺骗鉴别器
combined = Model(z, validity)
combined.compile(loss = 'binary_crossentropy', optimizer = optimizer)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 1024)             

# 建立图像目录

In [6]:
import os
os.mkdir('images')

# 图片取样

In [15]:
def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    
    # 正规化图片尺度到 0 - 1 的范围
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap = 'gray')
            axs[i, j].axis('off')
            cnt += 1
            
    fig.savefig('images/%d.png' % epoch)
    plt.close()

# 训练生成器与鉴别器

In [16]:
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
#init = tf.initialize_all_variables()

batch_size = 32
sample_interval = 200
epochs = 5000

# 读取 MNIST 数据 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 将尺度标准化至 -1 到 1 之间
x_train = x_train / 127.5 -1
x_train = np.expand_dims(x_train, axis = 3)

# 真的图片的答案判定为 1， 生成的图片判定为 0
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):
    # 建立鉴别器
    # 随机选择部分图片
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train[idx]
    
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    # 生成新图片
    gen_imgs = generator.predict(noise)
    
    # 训练鉴别器
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # 训练生成器
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = combined.train_on_batch(noise, valid)
    
    # 打印出过程
    print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
    
    # 产生输出样本
    if epoch % sample_interval == 0:
        sample_images(epoch)
        

0 [D loss: 0.665254, acc.: 62.50%] [G loss: 0.664062]


/opt/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.351580, acc.: 92.19%] [G loss: 0.610585]
2 [D loss: 0.319294, acc.: 93.75%] [G loss: 0.636436]
3 [D loss: 0.318599, acc.: 85.94%] [G loss: 0.716282]
4 [D loss: 0.312309, acc.: 89.06%] [G loss: 0.870275]
5 [D loss: 0.280096, acc.: 93.75%] [G loss: 0.978239]
6 [D loss: 0.254464, acc.: 96.88%] [G loss: 1.165205]
7 [D loss: 0.215865, acc.: 98.44%] [G loss: 1.343907]
8 [D loss: 0.170684, acc.: 100.00%] [G loss: 1.563853]
9 [D loss: 0.147659, acc.: 100.00%] [G loss: 1.605374]
10 [D loss: 0.122889, acc.: 100.00%] [G loss: 1.894214]
11 [D loss: 0.113721, acc.: 100.00%] [G loss: 2.171351]
12 [D loss: 0.093775, acc.: 100.00%] [G loss: 2.111836]
13 [D loss: 0.087448, acc.: 100.00%] [G loss: 2.296213]
14 [D loss: 0.070607, acc.: 100.00%] [G loss: 2.370162]
15 [D loss: 0.076657, acc.: 100.00%] [G loss: 2.496106]
16 [D loss: 0.060115, acc.: 100.00%] [G loss: 2.679325]
17 [D loss: 0.045555, acc.: 100.00%] [G loss: 2.755513]
18 [D loss: 0.052061, acc.: 100.00%] [G loss: 2.850075]
19 [D lo

151 [D loss: 0.407136, acc.: 81.25%] [G loss: 2.924300]
152 [D loss: 0.103667, acc.: 95.31%] [G loss: 3.697987]
153 [D loss: 0.098589, acc.: 95.31%] [G loss: 4.212307]
154 [D loss: 0.193718, acc.: 93.75%] [G loss: 4.274801]
155 [D loss: 0.203093, acc.: 90.62%] [G loss: 4.421772]
156 [D loss: 0.348502, acc.: 85.94%] [G loss: 3.545279]
157 [D loss: 0.129157, acc.: 96.88%] [G loss: 3.659213]
158 [D loss: 0.269826, acc.: 89.06%] [G loss: 2.643531]
159 [D loss: 0.148054, acc.: 95.31%] [G loss: 3.873649]
160 [D loss: 0.120363, acc.: 98.44%] [G loss: 4.226131]
161 [D loss: 0.259706, acc.: 84.38%] [G loss: 4.289206]
162 [D loss: 0.236089, acc.: 90.62%] [G loss: 4.370190]
163 [D loss: 0.135586, acc.: 96.88%] [G loss: 3.789001]
164 [D loss: 0.219838, acc.: 93.75%] [G loss: 3.275583]
165 [D loss: 0.232064, acc.: 90.62%] [G loss: 3.677835]
166 [D loss: 0.125982, acc.: 96.88%] [G loss: 3.686318]
167 [D loss: 0.258264, acc.: 85.94%] [G loss: 3.305478]
168 [D loss: 0.225682, acc.: 90.62%] [G loss: 3.

303 [D loss: 0.736661, acc.: 45.31%] [G loss: 0.896385]
304 [D loss: 0.740384, acc.: 46.88%] [G loss: 0.775817]
305 [D loss: 0.717263, acc.: 40.62%] [G loss: 0.767399]
306 [D loss: 0.774479, acc.: 39.06%] [G loss: 0.779054]
307 [D loss: 0.711992, acc.: 51.56%] [G loss: 0.947363]
308 [D loss: 0.710565, acc.: 46.88%] [G loss: 0.888022]
309 [D loss: 0.798437, acc.: 29.69%] [G loss: 0.777083]
310 [D loss: 0.740879, acc.: 45.31%] [G loss: 0.766964]
311 [D loss: 0.725460, acc.: 43.75%] [G loss: 0.773456]
312 [D loss: 0.777210, acc.: 39.06%] [G loss: 0.777521]
313 [D loss: 0.754123, acc.: 40.62%] [G loss: 0.815640]
314 [D loss: 0.773221, acc.: 43.75%] [G loss: 0.767578]
315 [D loss: 0.769128, acc.: 43.75%] [G loss: 0.757617]
316 [D loss: 0.721315, acc.: 46.88%] [G loss: 0.732996]
317 [D loss: 0.751961, acc.: 39.06%] [G loss: 0.731885]
318 [D loss: 0.698778, acc.: 48.44%] [G loss: 0.793346]
319 [D loss: 0.729218, acc.: 37.50%] [G loss: 0.803697]
320 [D loss: 0.856187, acc.: 21.88%] [G loss: 0.

...

4413 [D loss: 0.557340, acc.: 70.31%] [G loss: 1.049078]
4414 [D loss: 0.604808, acc.: 71.88%] [G loss: 0.988847]
4415 [D loss: 0.642930, acc.: 59.38%] [G loss: 0.962634]
4416 [D loss: 0.530332, acc.: 79.69%] [G loss: 1.042472]
4417 [D loss: 0.583721, acc.: 75.00%] [G loss: 1.017389]
4418 [D loss: 0.642932, acc.: 62.50%] [G loss: 1.101262]
4419 [D loss: 0.556529, acc.: 71.88%] [G loss: 0.989162]
4420 [D loss: 0.598992, acc.: 65.62%] [G loss: 1.009485]
4421 [D loss: 0.550915, acc.: 75.00%] [G loss: 0.936706]
4422 [D loss: 0.621953, acc.: 71.88%] [G loss: 0.931959]
4423 [D loss: 0.553276, acc.: 68.75%] [G loss: 1.022549]
4424 [D loss: 0.607927, acc.: 65.62%] [G loss: 1.132849]
4425 [D loss: 0.651953, acc.: 60.94%] [G loss: 1.010327]
4426 [D loss: 0.605356, acc.: 68.75%] [G loss: 1.007981]
4427 [D loss: 0.649234, acc.: 65.62%] [G loss: 1.026237]
4428 [D loss: 0.615006, acc.: 64.06%] [G loss: 1.071664]
4429 [D loss: 0.584457, acc.: 68.75%] [G loss: 0.883946]
4430 [D loss: 0.619743, acc.: 6

4557 [D loss: 0.611043, acc.: 65.62%] [G loss: 1.000567]
4558 [D loss: 0.591524, acc.: 70.31%] [G loss: 1.015517]
4559 [D loss: 0.663453, acc.: 62.50%] [G loss: 1.035222]
4560 [D loss: 0.536801, acc.: 79.69%] [G loss: 1.069507]
4561 [D loss: 0.660943, acc.: 60.94%] [G loss: 1.023659]
4562 [D loss: 0.610231, acc.: 67.19%] [G loss: 0.966546]
4563 [D loss: 0.554904, acc.: 71.88%] [G loss: 0.964180]
4564 [D loss: 0.596417, acc.: 70.31%] [G loss: 1.075419]
4565 [D loss: 0.633067, acc.: 65.62%] [G loss: 1.074288]
4566 [D loss: 0.580771, acc.: 64.06%] [G loss: 1.019877]
4567 [D loss: 0.540430, acc.: 78.12%] [G loss: 0.980109]
4568 [D loss: 0.560407, acc.: 73.44%] [G loss: 0.920997]
4569 [D loss: 0.588893, acc.: 67.19%] [G loss: 0.901616]
4570 [D loss: 0.628233, acc.: 62.50%] [G loss: 0.948429]
4571 [D loss: 0.645926, acc.: 54.69%] [G loss: 1.004043]
4572 [D loss: 0.592559, acc.: 60.94%] [G loss: 1.072839]
4573 [D loss: 0.549456, acc.: 73.44%] [G loss: 0.980793]
4574 [D loss: 0.636951, acc.: 5

4703 [D loss: 0.619421, acc.: 64.06%] [G loss: 1.140221]
4704 [D loss: 0.634009, acc.: 64.06%] [G loss: 0.995692]
4705 [D loss: 0.617224, acc.: 70.31%] [G loss: 1.061872]
4706 [D loss: 0.649875, acc.: 64.06%] [G loss: 0.962158]
4707 [D loss: 0.655190, acc.: 60.94%] [G loss: 0.958607]
4708 [D loss: 0.630614, acc.: 67.19%] [G loss: 0.998720]
4709 [D loss: 0.583367, acc.: 76.56%] [G loss: 1.081692]
4710 [D loss: 0.594065, acc.: 71.88%] [G loss: 0.886993]
4711 [D loss: 0.608817, acc.: 71.88%] [G loss: 0.939272]
4712 [D loss: 0.576144, acc.: 70.31%] [G loss: 0.986665]
4713 [D loss: 0.559014, acc.: 78.12%] [G loss: 0.958105]
4714 [D loss: 0.557358, acc.: 73.44%] [G loss: 0.985178]
4715 [D loss: 0.608256, acc.: 67.19%] [G loss: 0.998695]
4716 [D loss: 0.576334, acc.: 65.62%] [G loss: 1.077189]
4717 [D loss: 0.634526, acc.: 60.94%] [G loss: 1.192191]
4718 [D loss: 0.619906, acc.: 60.94%] [G loss: 1.125719]
4719 [D loss: 0.729256, acc.: 53.12%] [G loss: 0.970296]
4720 [D loss: 0.592402, acc.: 6

4849 [D loss: 0.679724, acc.: 57.81%] [G loss: 1.052088]
4850 [D loss: 0.635992, acc.: 64.06%] [G loss: 1.014536]
4851 [D loss: 0.647020, acc.: 62.50%] [G loss: 0.987780]
4852 [D loss: 0.669502, acc.: 57.81%] [G loss: 1.038049]
4853 [D loss: 0.613885, acc.: 70.31%] [G loss: 0.958962]
4854 [D loss: 0.612372, acc.: 68.75%] [G loss: 1.006483]
4855 [D loss: 0.593063, acc.: 65.62%] [G loss: 0.980465]
4856 [D loss: 0.650893, acc.: 62.50%] [G loss: 1.032255]
4857 [D loss: 0.693050, acc.: 54.69%] [G loss: 0.855498]
4858 [D loss: 0.580554, acc.: 70.31%] [G loss: 0.951734]
4859 [D loss: 0.609595, acc.: 68.75%] [G loss: 0.900783]
4860 [D loss: 0.604729, acc.: 67.19%] [G loss: 0.954669]
4861 [D loss: 0.551958, acc.: 73.44%] [G loss: 0.909409]
4862 [D loss: 0.624595, acc.: 65.62%] [G loss: 0.941890]
4863 [D loss: 0.605632, acc.: 67.19%] [G loss: 0.883058]
4864 [D loss: 0.540672, acc.: 70.31%] [G loss: 0.984244]
4865 [D loss: 0.645419, acc.: 70.31%] [G loss: 0.919712]
4866 [D loss: 0.621508, acc.: 6

4993 [D loss: 0.596750, acc.: 67.19%] [G loss: 1.033483]
4994 [D loss: 0.533584, acc.: 75.00%] [G loss: 1.036085]
4995 [D loss: 0.657421, acc.: 53.12%] [G loss: 1.113153]
4996 [D loss: 0.609070, acc.: 67.19%] [G loss: 1.076535]
4997 [D loss: 0.648282, acc.: 60.94%] [G loss: 1.037758]
4998 [D loss: 0.523980, acc.: 73.44%] [G loss: 0.989046]
4999 [D loss: 0.614633, acc.: 62.50%] [G loss: 1.005471]
